### BUSINESS CHALLENGE
#### Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
Company name and their primary website will be provided

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        
        soup = BeautifulSoup(self.body, 'html.parser')
        
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')]
        
        # get valid links only - if link is not None
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
website = Website("https://www.amdocs.com/")

## Extract useful links
1) Use gpt-4o-mini to read the links on a webpage (one shot prompting)
2) figure out which links are relevant
3) respond in structured JSON

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
Your task is to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages etc..\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "About page", "url": "https://full.url/goes/here/about"},
        {"type": "Careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
        ,response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [8]:
len(website.links)

281

In [9]:
get_links('https://www.amdocs.com/')

{'links': [{'type': 'About page', 'url': 'https://www.amdocs.com/about'},
  {'type': 'Careers page', 'url': 'https://jobs.amdocs.com/careers'},
  {'type': 'Our Purpose', 'url': 'https://www.amdocs.com/about/our-purpose'},
  {'type': 'Our Customers',
   'url': 'https://www.amdocs.com/about/our-customers'},
  {'type': 'Corporate Social Responsibility',
   'url': 'https://www.amdocs.com/about/corporate-social-responsibility'},
  {'type': 'Products and Services',
   'url': 'https://www.amdocs.com/products-services'},
  {'type': 'News and Press', 'url': 'https://www.amdocs.com/news-press'}]}

## make the brochure!

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()

    # only useful links from landing webpae
    links = get_links(url)
    # print("Found links:", links)
    
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [11]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs etc.. if you have the information."

In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20000] # Truncate if more than 20,000 characters
    return user_prompt

In [13]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [14]:
create_brochure("Amdocs", "https://www.amdocs.com/")

# Welcome to Amdocs: Where Technology Meets Magic! 🎩✨

## One Amazing Company

At **Amdocs**, we help those who build the future to make it *amazing*. With over 30,000 employees touching the lives of billions every day, we're the wizards behind the scenes of the connected world. Imagine if Gandalf and Einstein teamed up in tech—you're halfway there!

### Our Mission

Driving the digital society forward with jaw-dropping technology (and a sprinkle of pixie dust). Whether it’s preparing for the 5G explosion or pulling rabbits out of hats using generative AI (our very own `amAIz` Platform), we’re here to transform your wildest dreams into reality.

---

## Who Do We Work With?

**Big Shots of Telecom & Media:** We're talking about names like AT&T, BT, Comcast, and T-Mobile—just to name a few. We’ve got 350+ communications & service provider customers in over 85 countries! 🌍 And yes, we fill their days with content customers *actually* love. 

### 5G? You Bet!

With our cutting-edge 5G Experience Lab, we’re making today's impossible tomorrow's possible! We're like the Prometheus of telecommunications (minus the liver-pecking from eagles).

---

## What Makes Amdocs, Amdocs?

- **Global Impact:** Our solutions help over a billion consumers feel like VIPs. 🌟
- **Revenue in 2023:** A dazzling **$4.89 billion**. Not too shabby, huh?
- **Corporate Social Responsibility:** We don’t just collect awards like Pokémon cards, we actively make a difference in lives across social, economic, and environmental realms.

---

## Jumpstart Your Career

**Careers with Amdocs:** Why work at average when you can make things amazing? 🚀 Join us and embark on a journey that's gratifying, dynamic, and downright fun. We have spots to fill for tech enthusiasts, marketing mavens, and everything in between! 

### Our Company Culture

- **Imagine & Innovate:** At Amdocs, ideas aren’t just shared—they’re amplified.
- **Teamwork Makes the Dream Work:** Join a team where collaboration is key and every voice counts (even the one suggesting we bring back VHS tapes for nostalgic reasons).
- **Make it Awesome:** Whether you’re automating processes or launching a new product, mediocrity isn’t in our vocabulary.

---

## Why Choose Amdocs?

- **Innovative Solutions:** From AI to clouds—yeah, we’ve got it all covered. Not the weather kind, though. ☁️
- **Work & Play:** Flexible work arrangements allowing you to balance your inner workaholic with your couch-loving self. 
- **Learning & Growth:** The only stairs to climb are those taking you to success. And hey, who doesn’t love a good metaphorical climb?

---

Join us at Amdocs where we don’t just dream of a connected world, we build it! Ready to take the plunge? Head over to our **[Careers Page](#)** to start the adventure of a lifetime! 🚀🌐

### Contact Us
Got questions? Want to reach out? Don’t be shy; slide into our DMs or fill out our contact form. We promise we won't ghost you!

---

**Let’s make amazing things happen together!** 🌟✌️

In [15]:
create_brochure("HuggingFace", "https://huggingface.co/")

# Welcome to Hugging Face: The AI Playground of the Future! 🌟

---

## Who Are We? 🤔

At **Hugging Face**, we're assembling the Avengers of Artificial Intelligence—one commit at a time! As the ultimate AI community, we’re on a heroic mission to **democratize good machine learning**. No world-conquering plans here, just good vibes and even better models!

---

## What Do We Offer? 💻

Here at Hugging Face, you can:

- **No Models Left Behind**: Browse over **400k+ machine learning models**. Treat them like Pokémon—gotta catch ‘em all! 
- **Quench Your Data Thirst**: Dive into our treasure trove of **100k+ datasets!** Just don’t get confused and try to drink them. 💧
- **Space Invaders**: Create and share your application demos in our vibrant ML **Spaces**. Run your creations with a side of creativity!

---

## Our Customer Base: Who's Using Us? 🏢

We’re not just a small corner of the internet; we’ve partnered with heavyweights! Over **50,000 organizations** such as **Amazon, Microsoft, and Google** are hugging us tightly. Did we mention there’s room on this big hug for you too?

---

## Looking for a Job? Join Our Hug! 🤗

We’re always on the hunt for new talent! With a team vibe that’s part **scientist**, part **creative genius**, and a sprinkle of **tree-hugger**, we promise every day here feels like a passionate jam session. 

### Open Positions:

1. **ML Wizard 🧙‍♂️**: Seeking those who have mastered the secrets of machine learning spells.
2. **Data Guru 📊**: If you can conjure up numbers even in your sleep, we need you!
3. **Technical Communicator ✍️**: Your mission—make complex algorithms sound as fun as a cat video.

Join us, and let’s build something amazing!

---

## Our Culture: Hugging Faces, Not Just Algorithms! 🥳

Life at Hugging Face is a blend of hard work, playful banter, and an occasional game of AI trivia. Our **company culture**? Think inclusive spaces invited to the party:

- **Equal Opportunity for All**: A commitment to diversity, because good ideas come from all sources! 🌍
- **Feedback is Food for Growth**: Bring snacks—and feedback! Everyone’s voice has value, even if it’s just for the brownie recipe.
- **Work Hard, Play Hard**: Coffee breaks often turn into mini hackathons. If you overhear someone talking in code, don’t worry—they’re just having fun!

---

## Pricing Plans: Choose Your Adventure! 💸

We're transparent like the best kind of AI:
- **Free Forever**: Just like that friend who overstays their welcome but is still fun to have around.
- **Pro Account for $9/month**: More features to help your ML game—think of it as the deluxe pizza upgrade.
- **Enterprise**: Starting at $20/user/month for safety, security, and support... like a big, warm digital hug every month.

---

## Join the Hugging Revolution! 🚀

Whether you're a customer, an investor, or potential team member, we invite you to join our community. With Hugging Face, you’re not just along for the ride; you’re building the future of AI with us—**hugs included**!

For more information or to dive into the world of machine learning, visit [Hugging Face](https://huggingface.co). 

## Ready, Set, Hug! 🤗

--- 

*Remember: Life is short; make sure your model is long!*